<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import get_custom_objects
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [112]:
df=pd.read_csv("/content/updated_dataset.csv")

In [113]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [114]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [115]:

X = df[features].values
y = df[targets].values

In [116]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [117]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [118]:
hours_tensor = tf.convert_to_tensor(df['Hour'].values, dtype=tf.float32)


In [119]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)

In [120]:
def custom_loss_fixed(y_true, y_pred):
    batch_size = tf.shape(y_true)[0]
    batch_hours = tf.gather(hours_tensor, tf.range(batch_size))  # Use hours_tensor instead of hours

    nighttime_mask = tf.logical_or(tf.less_equal(batch_hours, 8), tf.greater_equal(batch_hours, 18))
    nighttime_mask = tf.cast(nighttime_mask, tf.float32)

    mse_loss = tf.keras.losses.MSE(y_true, y_pred)
    penalty_factor = 10.0
    weighted_loss = mse_loss * (1 + (penalty_factor - 1) * nighttime_mask)

    return tf.reduce_mean(weighted_loss)




In [121]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [122]:
model.compile(optimizer='adam', loss=custom_loss_fixed, metrics=['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [123]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 704192.3125 - mae: 199.1628 - val_loss: 325530.5938 - val_mae: 133.8725
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 334734.8750 - mae: 125.7118 - val_loss: 279939.6562 - val_mae: 108.6224
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 299002.0938 - mae: 108.8554 - val_loss: 248517.8281 - val_mae: 96.4577
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 259783.1406 - mae: 100.1180 - val_loss: 245309.7969 - val_mae: 93.8147
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 266457.5000 - mae: 99.4974 - val_loss: 248050.5938 - val_mae: 95.7746
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 251981.0781 - mae: 95.4579 - val_loss: 245922.8125 - val_mae: 92.4991
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 254925.1562 - mae: 95.7848 - val_loss: 233114.1250 - val_mae: 87.5705
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 245676.5625 - mae: 93.07

In [124]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [125]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [126]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour       DNI_A
6056      9   10     8  259.199219
5556      8   20    12  326.159363
5990      9    7    14  289.042267
7674     11   16    18   53.463699
3319      5   19     7  159.440567
...     ...  ...   ...         ...
6006      9    8     6   20.301859
8063     12    2    23   -5.241169
14        1    1    14  522.838074
5306      8   10     2    0.377983
8524     12   22     4   -0.516996

[100 rows x 4 columns]


In [127]:
model.save("solar_irradiance_predictions.keras")


model = keras.models.load_model(
    "solar_irradiance_predictions.keras",
    custom_objects={"custom_loss_fixed": custom_loss_fixed},
    safe_mode=False  # Allows deserialization of custom objects
)





In [128]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [129]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-129-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [130]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [131]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [132]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[0.         0.66068578 1.49168441 ... 0.18181818 7.14951514 0.22222222]
 [0.04347826 0.61124863 0.91899833 ... 0.18181818 4.75523477 0.11111111]
 [0.08695652 0.64239392 0.56288627 ... 0.18181818 2.66447391 0.33333333]
 ...
 [0.91304348 0.64660538 0.86204352 ... 0.18181818 6.17242166 0.11111111]
 [0.95652174 0.58637089 0.5889547  ... 0.18181818 0.47375851 0.11111111]
 [1.         0.64982638 0.72627768 ... 0.18181818 3.12254577 0.33333333]]


In [133]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 15] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [134]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      29.738356  1035.750532          0.497476   
1    1      3     1      28.126705  1018.569950          0.377762   
2    1      3     2      29.142042  1007.886588          0.273224   
3    1      3     3      27.607819  1007.773785          0.118153   
4    1      3     4      29.355007  1001.012570          0.307664   
5    1      3     5      28.985646  1024.710290          0.189662   
6    1      3     6      28.505506  1010.754263          0.289048   
7    1      3     7      29.981542  1005.345113          0.311924   
8    1      3     8      29.507550  1017.094996          0.271512   
9    1      3     9      28.702963  1008.341810          0.206139   

   Cloud Type_A       DNI_A  
0             2    0.000000  
1             1    0.000000  
2             3    0.000000  
3             1    0.000000  
4             3    0.000000  
5             1  141.291733  
6             1   26.454313  
